# Analysis with `crawl`

`crawl` has been design with BigQuery in mind as a target for data analysis. It's implementation is flexible, so it shouldn't be hard to get it working with other tools as well. This notebook is a tutorial on using BigQuery to analyze your crawl data. I'll assume you've run a crawl, and uploaded your data to BigQuery.

The [Google Cloud SDK](https://cloud.google.com/sdk/) is the easiest way to do this. If you haven't installed the SDK, you will need to install and configure it to run any analysis from the command line (or notebooks, like this one). You can also use SQL directly from the BigQuery interface.

To make things easier, we'll use the BigQuery Jupyter extension. Most of the analysis here will just use BigQuery's [Standard SQL](https://cloud.google.com/bigquery/docs/reference/standard-sql/).

In [2]:
%load_ext google.cloud.bigquery

## Basic query

If you want to know about a single page, or a group of pages, a simple query will answer most questions. The crawler outputs one record for each page it crawls

For instance, you can query the `StatusCode` of individual pages:

In [18]:
%%bigquery
SELECT Address.Full,
       Status,
       Depth
FROM crawl.distilled
ORDER BY Depth ASC
LIMIT 5

,Full,Status,Depth
0,https://www.distilled.net/,200 OK,0
1,https://www.distilled.net/store/landing/,301 Moved Permanently,1
2,https://www.distilled.net/store/profile/login/...,Blocked by robots.txt,1
3,https://www.distilled.net/store/profile/register/,200 OK,1
4,https://www.distilled.net/store/profile/passwo...,200 OK,1


If you're running a SQL query in the BigQuery interface, you can then export this table to Sheets or DataStudio.

_NB: when running queryies in a notebook, the object returned by the `%%bigquery` invocation is a Pandas dataframe. That doesn't matter much if it's anonymous. We can give it a name do that by adding a variable name to the invocation, like this:_

In [19]:
%%bigquery titles
SELECT Address.Full AS URL,
       Title
FROM crawl.distilled
WHERE StatusCode = 200 AND
      Address.Path LIKE "/resources/%"
ORDER BY Depth ASC

,URL,Title
0,https://www.distilled.net/resources/,Online Marketing Blog and Training Resources |...
1,https://www.distilled.net/resources/3-seo-spli...,3 SEO Split Tests You Should Try | Distilled
2,https://www.distilled.net/resources/posts/,"Distilled Blog | Online Marketing, Search, SEO..."
3,https://www.distilled.net/resources/videos/,HD Video Footage from Distilled's SearchLove C...
4,https://www.distilled.net/resources/creative-i...,Creative Inspiration: Content We Enjoyed this ...
5,https://www.distilled.net/resources/features/,Big Brand Reports and Training Guides from Dis...
6,https://www.distilled.net/resources/technical-...,Technical SEO Audit Checklist for Human Beings...
7,https://www.distilled.net/resources/how-to-be-...,How to be positive at work and make your life ...
8,https://www.distilled.net/resources/announcing...,Announcing Full-Funnel Testing - testing SEO a...
9,https://www.distilled.net/resources/how-to-mak...,How to Make a Histogram using Google Sheets | ...


In [22]:
titles.describe() # titles is now the table resulting from that query

,URL,Title
count,1254,1254
unique,1254,928
top,https://www.distilled.net/resources/be-your-ow...,"Distilled Blog | Online Marketing, Search, SEO..."
freq,1,213


## Summarizing data

You may want to summarize your crawl on some dimension. Here are some great candidates for summarizing:

- Group by `Status` to see whether the crawl contains non-200 status codes.
- Group by `Depth` for a histogram of page depths in the crawl.
- Group by `BodyTextHash` to find pages with (nearly-)identical body content.

Let's see what they look like:

### Status

In [29]:
%%bigquery
SELECT Status,
       COUNT(*) AS N
FROM crawl.distilled
GROUP BY Status
ORDER BY Status ASC

,Status,N
0,200 OK,3782
1,301 Moved Permanently,285
2,302 Moved Temporarily,155
3,404 Not Found,139
4,Blocked by robots.txt,2526


### Depth

In [30]:
%%bigquery
SELECT Depth,
       COUNT(*) AS N
FROM crawl.distilled
GROUP BY Depth
ORDER BY Depth ASC
LIMIT 5

,Depth,N
0,0,1
1,1,31
2,2,445
3,3,1394
4,4,1448


### BodyTextHash

When the crawler processes a page, it computes a unique value `BodyTextHash` based on all of the text nodes within the `<body>` tag. Think of this as an identifier for the text content of the page. If two pages share the same `BodyTextHash`, it is likely they are duplicates.

In [37]:
%%bigquery
SELECT BodyTextHash,
       ARRAY_AGG(Address.Full) AS Instances,
       COUNT(*) AS N
FROM crawl.distilled
WHERE BodyTextHash IS NOT NULL AND
      StatusCode = 200
GROUP BY BodyTextHash
ORDER BY N DESC
LIMIT 5

,BodyTextHash,Instances,N
0,s8KMOPbp341U4YFWdmzF1iwG+beJLnqM0jrBrswuExPnbJ...,"[https://www.distilled.net/, https://www.disti...",4
1,QUPuCxslts+PPl42gupqLIlA+Iy2ykEnm8Gagw2s9fXjSp...,[https://www.distilled.net/blog/bacon-mouthwas...,2
2,CL0DCWRCp4i3OwilAz3bL5tcVCzH7Zr7iAG6MteTpYjKeQ...,[https://www.distilled.net/blog/distilled/a-re...,2
3,vTj1A2EKDGMxwKjSEpBeYkch0zZNfyL5qiVK6dPqqBjvwX...,[https://www.distilled.net/events/searchlove-b...,2
4,tuOR1SJRsm+/pAnLIeh8Yst45YBNCe+08hRlKphFT6uJxK...,[https://www.distilled.net/events/linklove-lon...,2


## Accessing nested data with UNNEST

Some characteristics of a page can differ widely. For instance, all pages should have a single title tag. If they happen to have more than that, we only concern ourselves with the first. On the other hand, pages can have any number of links — from none to many — and we concern ourselves with all of them. Links aren't the only example of this. `hreflang` tags are another example. In fact, this applies to metadata about the crawler's request as well. There are a variable number of headers returned by the server.

BigQuery handles this possibility with _repeated fields_.

## Understanding relationships with JOIN

Repeated fields let us answer many common questions, but not all of them. Often, once we've established that a relationship between two pages exists, we need to understand something about the _other_ page. Data about the source page is easy to access — all relevant fields are immediately available. But if a page links to another, how do we get the StatusCode field for the target page?

This requires a maneuver called a "self-join". It can be a bit hard to wrap your head around at first. However, once you understand this pattern, most other questions you have can be answered this way.

I'll present a sample query and then explain.

## Analysis with Python

### Do you really need Python?

For most analyses, SQL should be sufficient. If you structure your queries wisely, it will also be efficient. However, there are some computations SQL will be unable to perform. For instance, anything involving recursion isn't happening in SQL. But think long and hard about whether you really need recursion.

For instance — if you want to know where redirect chains are, do you really need to see the whole chain of redirects? That would require recursion. Or could you get away with having a list of URLs that redirect to pages with a 3XX status code? That can be done without recursion.

If you're convinced you need to use Python to process your data set, well...

### OK, here's an example.

Imagine that our data set does _not_ include `Depth`. Calculating the distance of a page from the some starting page — usually the home page — would then require treating the entire crawl as a graph. The links between pages are the edges of this graph. We would perform a breadth-first search of this graph, incrementing a depth variable each time we complete a "level" without finding the page we're looking for.

This only requires two steps: first, execute a named query to associate the query results with a Python variable. Then, do something with that variable.

The first line in the next box is `%%bigquery link_graph`. That means the result of the query will be stored in the Python variable `link_graph` in subsequent cells.

In [24]:
%%bigquery link_graph
SELECT source.Address.Path AS SourcePath,
       target.Address.Path AS TargetPath
FROM crawl.distilled AS source, UNNEST(Links) AS target
WHERE target.Address.Host = "www.distilled.net"

,SourcePath,TargetPath
0,/manifesto/,/
1,/events/searchlove-boston/schedule/,/events/searchlove-boston/schedule/
2,/events/searchlove-boston/schedule/,/events/searchlove-boston/schedule/
3,/events/searchlove-boston/schedule/,/
4,/events/searchlove-boston/schedule/,/events/searchlove-boston/schedule/
5,/events/searchlove-boston/schedule/,/events/searchlove-boston/
6,/events/searchlove-boston/schedule/,/events/searchlove-boston/schedule/
7,/events/searchlove-boston/schedule/,/store/profile/password/reset/
8,/events/searchlove-boston/schedule/,/store/profile/register/
9,/events/searchlove-boston/schedule/,/events/searchlove-boston/schedule/


Now we execute the breadth-first-search, keeping track of which pages we've seen and how many levels we've traversed. When we see a page for the first time, we put an entry into a dictionary. The key is the URL and the value is the current depth.

This will take approximately three years to complete — that's why the crawler includes `Depth` as a field by default!

In [25]:
from collections import deque

def edges(graph, node):
    for edge in graph.loc[graph["SourcePath"] == node].itertuples():
        yield edge

def breadth_first_search(graph, source):
    depth = 0
    depths = {source: depth}
    q = deque()
    q.append(source)
    while q:
        source = q.pop()
        depth = depths[source]
        for edge in edges(graph, source):
            target = edge.TargetAddress
            if target not in depths:
                depths[target] = depth + 1
                q.append(target)
    return depths
    
depths = breadth_first_search(link_graph, "https://www.distilled.net/")

OK, maybe more like three minutes. To prove it works, an example result:

In [8]:
depths['https://www.distilled.net/resources/posts/marketing/']

6

Still, that's a while for a relatively small site! How many pages are we talking about?

In [6]:
%%bigquery
SELECT COUNT(Address)
FROM crawl.distilled

,f0_
0,6887


Thanks for reading, and Happy Crawling.